In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from os import listdir, getcwd, chdir
from os.path import isfile, join
import json

In [36]:
# Paths to where I store the data on my computer
msd_data_path = '/Users/nelsschimek/Documents/nancelab/Data/msd_data_age/'
feature_data_path = '/Users/nelsschimek/Documents/nancelab/Data/feature_data_age/'

json_filepath = '/Users/nelsschimek/Documents/becklab/mpt-statistical-testing/15_models_10_percent.json'

In [22]:
msd_files = [f for f in listdir(msd_data_path) if isfile(join(msd_data_path, f)) and '.csv' in f]
feature_files = [f for f in listdir(feature_data_path) if isfile(join(feature_data_path, f)) and '.csv' in f]
print(len(msd_files))
print(len(feature_files))

74
75


In [41]:
# Load the json file

f1 = open(json_filepath, 'r')
json_data = json.load(f1)
f1.close()

In [45]:
#check the keys
print(len(json_data.keys()))
for key in json_data.keys():
    print(key)

75
/features_P14_40nm_s1_v1.csv
/features_P14_40nm_s1_v2.csv
/features_P14_40nm_s1_v3.csv
/features_P14_40nm_s1_v4.csv
/features_P14_40nm_s1_v5.csv
/features_P14_40nm_s2_v1.csv
/features_P14_40nm_s2_v2.csv
/features_P14_40nm_s2_v3.csv
/features_P14_40nm_s2_v4.csv
/features_P14_40nm_s2_v5.csv
/features_P14_40nm_s3_v1.csv
/features_P14_40nm_s3_v2.csv
/features_P14_40nm_s3_v3.csv
/features_P14_40nm_s3_v4.csv
/features_P14_40nm_s3_v5.csv
/features_P21_40nm_s1_v1.csv
/features_P21_40nm_s1_v2.csv
/features_P21_40nm_s1_v3.csv
/features_P21_40nm_s1_v4.csv
/features_P21_40nm_s1_v5.csv
/features_P21_40nm_s2_v1.csv
/features_P21_40nm_s2_v2.csv
/features_P21_40nm_s2_v3.csv
/features_P21_40nm_s2_v4.csv
/features_P21_40nm_s2_v5.csv
/features_P21_40nm_s3_v1.csv
/features_P21_40nm_s3_v2.csv
/features_P21_40nm_s3_v3.csv
/features_P21_40nm_s3_v4.csv
/features_P21_40nm_s3_v5.csv
/features_P28_40nm_s1_v1.csv
/features_P28_40nm_s1_v2.csv
/features_P28_40nm_s1_v3.csv
/features_P28_40nm_s1_v4.csv
/features_P

In [23]:
# Below is a list of features that we typically use in our analysis. Frames is commented out as it is not a statistical or geometric feature of the trajectory
# However it could be interesting to see if the number of frames a track spans is related to data quality.
feature_list = [
    'alpha', # Fitted anomalous diffusion alpha exponenet
    'D_fit', # Fitted anomalous diffusion coefficient
    'kurtosis', # Kurtosis of track
    'asymmetry1', # Asymmetry of trajecory (0 for circular symmetric, 1 for linear)
    'asymmetry2', # Ratio of the smaller to larger principal radius of gyration
    'asymmetry3', # An asymmetric feature that accnts for non-cylindrically symmetric pt distributions
    'AR', # Aspect ratio of long and short side of trajectory's minimum bounding rectangle
    'elongation', # Est. of amount of extension of trajectory from centroid
    'boundedness', # How much a particle with Deff is restricted by a circular confinement of radius r
    'fractal_dim', # Measure of how complicated a self similar figure is
    'trappedness', # Probability that a particle with Deff is trapped in a region
    'efficiency', # Ratio of squared net displacement to the sum of squared step lengths
    'straightness', # Ratio of net displacement to the sum of squared step lengths
    'MSD_ratio', # MSD ratio of the track
#     'frames', # Number of frames the track spans
    'Deff1', # Effective diffusion coefficient at 0.33 s
    'Deff2', # Effective diffusion coefficient at 3.3 s
    'Mean alpha', 
    'Mean D_fit', 
    'Mean kurtosis', 
    'Mean asymmetry1', 
    'Mean asymmetry2',
    'Mean asymmetry3', 
    'Mean AR',
    'Mean elongation', 
    'Mean boundedness',
    'Mean fractal_dim', 
    'Mean trappedness', 
    'Mean efficiency',
    'Mean straightness', 
    'Mean MSD_ratio', 
    'Mean Deff1', 
    'Mean Deff2',
    ]

#### Ok, lets read in a feature file and make sure it lines up with the json data

In [49]:
#load the data
features_p35_s3_v3 = pd.read_csv(feature_data_path + 'features_P35_brain_2_slice_3_vid_3.csv')
print(features_p35_s3_v3.shape)

(817, 92)


In [55]:
quality_p35_s3_v3 = json_data['/features_P35_brain_2_slice_3_vid_3.csv']
len(quality_p35_s3_v3)

683

#### First issue - the json file has a different number of tracks than the feature file. This is likely because the feature file has been filtered to remove tracks have NAs in them. Lets remove NAs and compare again


In [57]:
p35_s3_v3_filtered = features_p35_s3_v3[feature_list + ['Track_ID', 'X', 'Y', 'frames']]
removed_nans = p35_s3_v3_filtered[~p35_s3_v3_filtered[list(set(feature_list) - set(['Deff2', 'Mean Deff2']))].isin([np.nan, np.inf, -np.inf]).any(1)]
removed_nans.shape

(683, 36)

#### Both now have 683 - perfect! That line of code to remove nans is what always happened before putting the data into an ML model when the quality metric was being estimated. 